# Linearity in Finance

In [1]:
import cvxpy as cp
import numpy as np
from scipy import linalg, stats

## The Capital Asset Pricing Model

In [2]:
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)
print(f"beta: {beta}, alpha: {alpha}")

beta: 0.5077431878770808, alpha: -0.008481900352462384


## Multivariate Linear Regression Of Factor Models

In [3]:
# Generate some sample data
num_periods = 20
all_values = np.array([np.random.random(8) for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0]  # First column values as Y
x_values = all_values[:, 1:]  # All other values as X

## Linear Optimization

### A Maximization Example With Linear Programming

In [4]:
x = cp.Variable(name="x", integer=True)
y = cp.Variable(name="y", integer=True)

objective = cp.Maximize(3 * x + 2 * y)
constraints = [2 * x + y <= 100, x + y <= 80, x <= 40, x >= 0, y >= 0]
problem = cp.Problem(objective, constraints)
problem.solve()

np.float64(180.0)

In [5]:
print(f"Status: {problem.status}")
print(f"Maximization Results: {problem.value}")
for variable in problem.variables():
    print(f"{variable} = {variable.value}")

Status: optimal
Maximization Results: 180.0
x = 20.0
y = 60.0


### A Minimization Example With Integer Programming


In [6]:
x1 = cp.Variable(name="x1", integer=True)
x2 = cp.Variable(name="x2", integer=True)
x3 = cp.Variable(name="x3", integer=True)
quantities = cp.hstack([x1, x2, x3])
costs = cp.Constant([500, 350, 450])
fixed = cp.Constant([4000, 2000, 6000])
costs @ quantities + fixed

Expression(AFFINE, UNKNOWN, (3,))

In [7]:
is_orders = cp.Variable(3, name="is_orders", boolean=True)
(costs @ quantities + fixed) @ is_orders

Expression(UNKNOWN, UNKNOWN, ())

In [8]:
objective = cp.Minimize(cp.sum(costs @ quantities + fixed @ is_orders))
constraints = [
    x1 >= 30 * is_orders[0],
    x1 <= 100 * is_orders[0],
    x2 >= 30 * is_orders[1],
    x2 <= 90 * is_orders[1],
    x3 >= 30 * is_orders[2],
    x3 <= 70 * is_orders[2],
    x1 + x2 + x3 == 150,
]

problem = cp.Problem(objective, constraints)
problem.solve()


np.float64(66500.0)

## Solving Linear Equations Using Matrices

In [9]:
A = np.array([[2, 1, 1], [1, 3, 2], [1, 0, 0]])
B = np.array([4, 5, 6])
print(np.linalg.solve(A, B))

[  6.  15. -23.]


### The LU decomposition


In [10]:
# Define A and B
A = np.array([[2.0, 1.0, 1.0], [1.0, 3.0, 2.0], [1.0, 0.0, 0.0]])
B = np.array([4.0, 5.0, 6.0])

# Perform LU decompositiopn
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
print(x)

[  6.  15. -23.]


In [11]:
P, L, U = linalg.lu(A)

print("P=\n", P)
print("L=\n", L)
print("U=\n", U)

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


### The Cholesky Decomposition

In [12]:
A = np.array(
    [
        [10.0, -1.0, 2.0, 0.0],
        [-1.0, 11.0, -1.0, 3.0],
        [2.0, -1.0, 10.0, -1.0],
        [0.0, 3.0, -1.0, 8.0],
    ]
)
B = np.array([6.0, 25.0, -11.0, 15.0])

L = np.linalg.cholesky(A)
print(L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [13]:
print(np.dot(L, L.T.conj()))  # A=L.L^*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [14]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y
print(x)

[ 1.  2. -1.  1.]


In [15]:
print(np.asmatrix(A) * np.asmatrix(x).T)  # B=Aximport numpy as npimport numpy as np

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


### The QR decomposition

In [16]:
A = np.array([[2.0, 1.0, 1.0], [1.0, 3.0, 2.0], [1.0, 0.0, 0]])
B = np.array([4.0, 5.0, 6.0])

Q, R = linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = linalg.solve(R, y)  # Solve Rx=y
print(x)

[  6.  15. -23.]


## Solving With Other Matrix Algebra Methods


### The Jacobi Method


In [17]:
def jacobi_iter(A, B, n_iter=25, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for _ in range(n_iter):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1 :], x[i + 1 :])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [18]:
A = np.array(
    [
        [10.0, -1.0, 2.0, 0.0],
        [-1.0, 11.0, -1.0, 3.0],
        [2.0, -1.0, 10.0, -1.0],
        [0.0, 3.0, -1.0, 8.0],
    ]
)
B = np.array([6.0, 25.0, -11.0, 15.0])

x = jacobi_iter(A, B)
print(f"x = {x}")

x = [ 1.  2. -1.  1.]


### The Gauss-Seidel Method

In [19]:
def gauss_iter(A, B, n_iter=25, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A - L  # decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)

    for _ in range(n_iter):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)
        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [20]:
x = gauss_iter(A, B)
print(f"x = {x}")

x = [ 1.  2. -1.  1.]
